MODEL TRAINING


In [1]:
import pandas as pd

In [2]:
df2=pd.read_csv('data/breast-cancer.csv')

In [3]:
df2.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
#ID column will not be necessary for our model prediction,so drop it
df2=df2.drop('id',axis=1)

In [5]:
#Now to encode diagnosis column
df2["diagnosis"].dtype

dtype('O')

In [6]:
df2["diagnosis"] = df2["diagnosis"].astype(str).str.strip()


In [7]:
df2["diagnosis"].dtype


dtype('O')

In [8]:
#Split dataset into x and y
X=df2.drop('diagnosis',axis=1)
y=df2['diagnosis']

In [9]:

X

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [10]:
y

0      M
1      M
2      M
3      M
4      M
      ..
564    M
565    M
566    M
567    M
568    B
Name: diagnosis, Length: 569, dtype: object

In [11]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [12]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Label Encoding
from sklearn.preprocessing import LabelEncoder
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder', OrdinalEncoder(categories=[["B", "M"]])),
    ('scaler',StandardScaler())
    ]

)


preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [15]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [16]:
X_train.head()

,num_pipeline__radius_mean,num_pipeline__texture_mean,num_pipeline__perimeter_mean,num_pipeline__area_mean,num_pipeline__smoothness_mean,num_pipeline__compactness_mean,num_pipeline__concavity_mean,num_pipeline__concave points_mean,num_pipeline__symmetry_mean,num_pipeline__fractal_dimension_mean,num_pipeline__radius_se,num_pipeline__texture_se,num_pipeline__perimeter_se,num_pipeline__area_se,num_pipeline__smoothness_se,num_pipeline__compactness_se,num_pipeline__concavity_se,num_pipeline__concave points_se,num_pipeline__symmetry_se,num_pipeline__fractal_dimension_se,num_pipeline__radius_worst,num_pipeline__texture_worst,num_pipeline__perimeter_worst,num_pipeline__area_worst,num_pipeline__smoothness_worst,num_pipeline__compactness_worst,num_pipeline__concavity_worst,num_pipeline__concave points_worst,num_pipeline__symmetry_worst,num_pipeline__fractal_dimension_worst
0,-0.158032,1.262998,-0.174087,-0.212910,-0.107179,-0.380141,-0.029636,-0.108506,-0.716529,-0.581723,0.200904,0.152653,-0.008110,0.077084,-0.391803,-0.610709,-0.190150,-0.329493,-0.712932,-0.591084,0.143975,1.505035,0.031315,0.044762,-0.261784,-0.388747,0.194980,0.047467,-0.379281,-0.552506
1,-0.919404,-1.028057,-0.941355,-0.834150,-1.381996,-1.084917,-0.794176,-1.121838,-0.995625,-0.101365,-0.854119,-0.950339,-0.855416,-0.643535,-0.921428,-0.621712,-0.452777,-1.278622,-1.059075,-0.102634,-0.775437,-0.860226,-0.821958,-0.695851,-0.885775,-0.428762,-0.314498,-0.923457,-0.478646,0.386085
2,-0.811916,0.224687,-0.626922,-0.810151,3.367738,3.362908,1.976427,1.514967,3.009768,4.842270,0.353123,-0.092904,0.301544,-0.303057,0.671388,2.846662,0.929448,1.113915,4.969365,2.240851,-0.299732,0.114731,-0.267223,-0.567536,3.322935,3.812789,1.924399,2.196663,6.115303,4.725673
3,-0.889546,0.457179,-0.913674,-0.827771,0.397786,-0.857084,-1.029047,-1.027245,-0.693900,-0.199397,-0.460919,0.863669,-0.481604,-0.499937,-0.051634,-0.925637,-0.914435,-0.950147,0.839656,-0.712990,-0.843700,0.498372,-0.868902,-0.765792,-0.188627,-0.910813,-1.118819,-1.116591,0.150124,-0.698545
4,-0.952248,0.443636,-0.872586,-0.855719,1.922896,0.310701,0.205041,0.209136,0.362136,0.817339,-0.655183,0.405540,-0.559018,-0.548201,0.327936,-0.450871,-0.209467,-0.255417,-0.849368,-0.308014,-0.741306,1.516132,-0.624671,-0.664585,2.509595,0.083301,0.594148,0.418284,0.130577,0.624864


In [17]:
label_encoder=LabelEncoder()
y_train = label_encoder.fit_transform(y_train)  # Encode labels
y_test = label_encoder.transform(y_test)

In [18]:
print(y_train.dtype)
print(y_test.dtype)
print(y_train[:5])

int64
int64
[1 0 1 0 1]


In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import precision_score,accuracy_score,recall_score,f1_score

In [20]:
import numpy as np
def evaluate_model(true, predicted):
    precison = precision_score(true, predicted)
    recall = recall_score(true, predicted)
    f1score = f1_score(true, predicted)
    accuracy=accuracy_score(true,predicted)
    return precison,recall,f1score,accuracy

In [23]:
## Train multiple models
## Model Ecaluation
models={
    'DecisionTreeClassifier':DecisionTreeClassifier(),
    'GaussianNB':GaussianNB(),
    'SVC':SVC()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    precision,recall,f1score,accuracy=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("Precision",precision)
    print("Recall:",recall)
    print("F1 score",f1score)
    print('Accuracy',accuracy)

    
    print('='*35)
    print('\n')


DecisionTreeClassifier
Model Training Performance
Precision 0.890625
Recall: 0.9193548387096774
F1 score 0.9047619047619048
Accuracy 0.9298245614035088


GaussianNB
Model Training Performance
Precision 0.9032258064516129
Recall: 0.9032258064516129
F1 score 0.9032258064516129
Accuracy 0.9298245614035088


SVC
Model Training Performance
Precision 0.9833333333333333
Recall: 0.9516129032258065
F1 score 0.9672131147540983
Accuracy 0.9766081871345029


